In [ ]:
import pandas as pd
import eval
from tqdm.auto import tqdm

---
csv 파일 불러오기

In [ ]:
fnl_df_glowpick_prd_info = pd.read_csv("fnl_df_glowpick_prd_info_w_review.csv")
fnl_df_glowpick_prd_info = fnl_df_glowpick_prd_info.iloc[:, 1:]

In [ ]:
fnl_df_brd_w_prd = pd.read_csv("fnl_df_brd_w_prd.csv")
fnl_df_brd_w_prd = fnl_df_brd_w_prd.iloc[:, 1:]

for i in fnl_df_brd_w_prd.index:
    fnl_df_brd_w_prd.at[i, 'product_name'] = eval(fnl_df_brd_w_prd.loc[i, 'product_name'])
    fnl_df_brd_w_prd.at[i, 'product_link'] = eval(fnl_df_brd_w_prd.loc[i, 'product_link'])

In [ ]:
df_glowpick_prd = pd.read_csv("df_glowpick_prd.csv")
df_glowpick_prd = df_glowpick_prd.iloc[:, 1:]

df_glowpick_prd.columns = ['브랜드명_fix', '제품 id', '제품명_fix'] #구분을 위해 컬럼명 변경

---
DB 업로드를 위한 전처리, 정렬

In [ ]:
#브랜드 id와 브랜드 link값 채우기

for idx in tqdm(list(fnl_df_glowpick_prd_info.index)[:30000]):
    brd_name = fnl_df_glowpick_prd_info.loc[idx, '브랜드명']
    
    try:
        fnl_df_glowpick_prd_info.loc[idx, '브랜드 link'] = fnl_df_brd_w_prd.loc[fnl_df_brd_w_prd['brand_name'] == brd_name, 'brand_code']
    
    #브랜드명이 중복인 경우
    except ValueError:
        for i in list(fnl_df_brd_w_prd.index):
            if fnl_df_glowpick_prd_info.loc[idx, '제품 link'] in fnl_df_brd_w_prd.loc[i, 'product_link']:
                fnl_df_glowpick_prd_info.loc[idx, '브랜드 link'] = fnl_df_brd_w_prd.loc[i, 'brand_code']

In [ ]:
#bridge table의 제품 정렬 순서로 sorting
df_glowpick_prd_sort = pd.merge(df_glowpick_prd, fnl_df_glowpick_prd_info, how='inner', on='제품 id')

#브릿지 테이블에서 픽스된 정확한 브랜드명과 제품명을 사용하기 위해 재매핑
df_glowpick_prd_sort = df_glowpick_prd_sort.drop(['브랜드명'], axis = 'columns') #크롤링한 브랜드명 삭제
df_glowpick_prd_sort = df_glowpick_prd_sort.drop(['제품명'], axis = 'columns') #크롤링한 제품명 삭제

df_glowpick_prd_sort = df_glowpick_prd_sort.rename({'브랜드명_fix': '브랜드명'}, axis = 'columns') #브릿지 테이블에서 픽스된 브랜드명 사용
df_glowpick_prd_sort = df_glowpick_prd_sort.rename({'제품명_fix': '제품명'}, axis = 'columns') #브릿지 테이블에서 픽스된 제품명 사용

In [ ]:
df_glowpick_prd_sort = df_glowpick_prd_sort[columns] #컬럼 순서 변경

In [ ]:
#db에 올릴 테이블은 이미지 src 삭제
df_glowpick_prd_sort_drop_img_src = df_glowpick_prd_sort.drop('img_src')
df_glowpick_prd_sort_drop_img_src = df_glowpick_prd_sort_drop_img_src.reset_index(drop=True)
df_glowpick_prd_sort_drop_img_src.to_csv("fnl_df_glowpick_prd_info.csv")

df_glowpick_prd_sort = df_glowpick_prd_sort.reset_index(drop=True)
df_glowpick_prd_sort.to_csv("fnl_df_glowpick_prd_info_w_img_src.csv")

---
create tabel

---
DB upload

In [ ]:
import pymysql
from sqlalchemy import create_engine

In [ ]:
def db_connection(host_name='host_name'):
    host_url = "host_url"
    user_nm = "id # 아이디 입력
    passwd = "pw" # 패스워드 입력
    port_num = 0
    db_name = "db_name"
    conn = pymysql.connect(host=host_url, user=user_nm, passwd=passwd, port=port_num, charset='utf8', cursorclass=pymysql.cursors.DictCursor) # cursorclass=pymysql.cursors.DictCursor 추가     # db=db,
    return conn


def engine_upload(upload_df: object, host_name: object = 'ds', db_name: object = 'beauty_kr', table_name: object = 'ranking') -> object:
    host_url = "host_url"
    user_nm = "id" # 아이디 입력
    passwd = "pw"  # 패스워드 입력
    port_num = 0
    db_name = "db_name"
    engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
    engine_conn = engine.connect()
    upload_df.to_sql(table_name, engine_conn, if_exists='append', index=None)
    engine_conn.close()
    engine.dispose()

In [ ]:
engine_upload(host_name='ds', db_name='beauty_kr', table_name=f'glowpick_product_info', upload_df=fnl_df_glowpick_prd_info) 
# 필요한 스키마명, 테이블명만 변경해서 사용 